In [1]:
!pip install sparqlwrapper

    100% |████████████████████████████████| 348kB 1.7MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 3.4MB/s ta 0:00:011
  Running setup.py bdist_wheel for sparqlwrapper ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/c9/6b/9c/c69241fc9b429e0a0b32468dc6ba2e2b5974e8453948391d1d
Successfully built sparqlwrapper
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [ ]:
SELECT ?abstract WHERE {
    dbr:Hamburger dbo:abstract ?abstract
    filter(langMatches(lang(?abstract),"en"))
}

### Extracting Info about Hamburgers

In [17]:
### Get all Hamburger Country of Origin
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT * WHERE {
    dbr:Hamburger dbo:country ?what

}"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for r in results['results']['bindings']:
    print(r['what']['value'])

http://dbpedia.org/resource/United_States
http://dbpedia.org/resource/Germany


In [24]:
### What is a hamburger?
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT * WHERE {
    dbr:Hamburger dbo:type ?what

}"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for r in results['results']['bindings']:
    print(r['what']['value'])

http://dbpedia.org/resource/Main_course


In [27]:
### Get other main courses
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT * WHERE {
    ?what dbo:type dbr:Main_course
}"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for r in results['results']['bindings']:
    print(r['what']['value'])

http://dbpedia.org/resource/Hamburger
http://dbpedia.org/resource/Paella
http://dbpedia.org/resource/Crispy_pata
http://dbpedia.org/resource/Curacha
http://dbpedia.org/resource/Fish_head_casserole
http://dbpedia.org/resource/Wonton_noodles
http://dbpedia.org/resource/Cheeseburger
http://dbpedia.org/resource/Borani
http://dbpedia.org/resource/Dabu-dabu
http://dbpedia.org/resource/Freekeh
http://dbpedia.org/resource/Fasole_cu_cârnați
http://dbpedia.org/resource/Pasta_salad
http://dbpedia.org/resource/Brudet
http://dbpedia.org/resource/Panta_bhat
http://dbpedia.org/resource/Bacon_and_cabbage
http://dbpedia.org/resource/Marinara_sauce
http://dbpedia.org/resource/Goat_roti
http://dbpedia.org/resource/Fabada_asturiana
http://dbpedia.org/resource/Arroz_a_la_cubana
http://dbpedia.org/resource/Chermoula
http://dbpedia.org/resource/Chicken_fingers
http://dbpedia.org/resource/Mititei
http://dbpedia.org/resource/Tater_tots
http://dbpedia.org/resource/Doenjang_jjigae
http://dbpedia.org/resource/Dau

In [31]:
### Get other main courses
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>

SELECT * WHERE {
    ?what dct:subject dbc:American_cuisine
}"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for r in results['results']['bindings']:
    print(r['what']['value'])

http://dbpedia.org/resource/Apple_sauce
http://dbpedia.org/resource/Barbecue
http://dbpedia.org/resource/Chowder
http://dbpedia.org/resource/Fortune_cookie
http://dbpedia.org/resource/Hamburger
http://dbpedia.org/resource/Smelt_(fish)
http://dbpedia.org/resource/Taco
http://dbpedia.org/resource/Potato_wedges
http://dbpedia.org/resource/Vichyssoise
http://dbpedia.org/resource/Baconnaise
http://dbpedia.org/resource/Bull_roast
http://dbpedia.org/resource/Stuffed_ham
http://dbpedia.org/resource/Kansas_City-style_barbecue
http://dbpedia.org/resource/Cowboy_beans
http://dbpedia.org/resource/Pizza_in_the_United_States
http://dbpedia.org/resource/Corn_flakes
http://dbpedia.org/resource/Toaster_Strudel
http://dbpedia.org/resource/Pigs_in_blankets
http://dbpedia.org/resource/Pudding
http://dbpedia.org/resource/Baked_potato
http://dbpedia.org/resource/Cheese_fries
http://dbpedia.org/resource/Coleslaw
http://dbpedia.org/resource/Drunken_chicken
http://dbpedia.org/resource/Barbecue_in_the_United_St